## In these exercises, you will complete the first step toward the above goal: acquire and prepare the necessary Zillow data from the zillow database in the Codeup database server.

1. Acquire bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, and fips from the zillow database for all 'Single Family Residential' properties.
2. Using your acquired Zillow data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaningful; remember to document your process and decisions using markdown and code commenting where helpful.
3. Store all of the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe with no missing values in your wrangle.py file. Name your final function wrangle_zillow.

# Acquire

In [2]:
import pandas as pd 
import env
import os
from sklearn.model_selection import train_test_split
def get_db_url(database):
    return f'mysql+pymysql://{env.username}:{env.password}@{env.host}/{database}'

In [6]:
url=get_db_url('zillow')
    
SQL_query= """select id, parcelid,bathroomcnt, bedroomcnt,calculatedfinishedsquarefeet
        ,taxvaluedollarcnt,yearbuilt,taxamount, fips
         from properties_2017
         where propertylandusetypeid=261"""
    
filename=('zillow.csv')

directory='/Users/chellyannmoreno/codeup-data-science/regression-exercises/'

In [ ]:
df=pd.read_sql(SQL_query,url)

In [40]:
def get_data():
    if os.path.exists(directory+filename):
        df=pd.read_csv(filename)
        return df
    else:
        df=pd.read_sql(SQL_query,url)
        df.to_csv(filename,index=False)
        return df

In [41]:
df=get_data()

In [42]:
df.head()

,id,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,1,10759547,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,15,11193347,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,16,11215747,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,17,11229347,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,20,11324547,2.0,4.0,3633.0,296425.0,2005.0,6941.39,6037.0


# Prepare

In [25]:
df = df.rename(columns={
        'bedroomcnt': 'bedrooms',
        'bathroomcnt': 'bathrooms',
        'calculatedfinishedsquarefeet': 'area',
        'taxvaluedollarcnt': 'taxvalue',
        'fips': 'county'
    })

    # Filter out rows with large area
df = df[df.area < 25_000]

    # Drop rows with missing values
df = df.dropna()

    # Filter out rows with high tax value
taxvalue_threshold = df.taxvalue.quantile(.95)
df = df[df.taxvalue < taxvalue_threshold].copy()

    # Convert data types
df[['bedrooms', 'area', 'taxvalue', 'yearbuilt']] = df[['bedrooms', 'area', 'taxvalue', 'yearbuilt']].astype(int)

    # Map county codes to names
county_map = {6037: 'LA', 6059: 'Orange', 6111: 'Ventura'}
df.county = df.county.map(county_map)

In [26]:
df.shape

(2033172, 9)

In [27]:
df.head()

,id,parcelid,bathrooms,bedrooms,area,taxvalue,yearbuilt,taxamount,county
4,20,11324547,2.0,4,3633,296425,2005,6941.39,LA
6,31,11544747,4.0,3,1620,847770,2011,10244.94,LA
7,33,11585547,2.0,3,2077,646760,1926,7924.68,LA
11,62,11954547,0.0,0,1200,5328,1972,91.60,LA
14,97,12537947,0.0,0,171,6920,1973,255.17,LA


In [28]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,2033172.0,1.490903e+06,8.608481e+05,20.00,7.455758e+05,1490529.00,2.235772e+06,2.982282e+06
parcelid,2033172.0,1.304209e+07,4.344449e+06,10711725.00,1.167093e+07,12610664.50,1.404881e+07,1.681836e+08
bathrooms,2033172.0,2.141026e+00,8.351854e-01,0.00,2.000000e+00,2.00,3.000000e+00,2.000000e+01
bedrooms,2033172.0,3.257700e+00,8.967887e-01,0.00,3.000000e+00,3.00,4.000000e+00,1.600000e+01
area,2033172.0,1.751421e+03,7.258099e+02,1.00,1.242000e+03,1587.00,2.106000e+03,2.422000e+04
taxvalue,2033172.0,3.632401e+05,2.432811e+05,22.00,1.816560e+05,312745.00,4.917210e+05,1.202999e+06
yearbuilt,2033172.0,1.960267e+03,2.154476e+01,1801.00,1.949000e+03,1958.00,1.974000e+03,2.016000e+03
taxamount,2033172.0,4.506123e+03,2.842968e+03,6.34,2.456707e+03,3937.89,5.938602e+03,1.003793e+05


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2033172 entries, 4 to 2152862
Data columns (total 9 columns):
 #   Column     Dtype  
---  ------     -----  
 0   id         int64  
 1   parcelid   int64  
 2   bathrooms  float64
 3   bedrooms   int64  
 4   area       int64  
 5   taxvalue   int64  
 6   yearbuilt  int64  
 7   taxamount  float64
 8   county     object 
dtypes: float64(2), int64(6), object(1)
memory usage: 155.1+ MB


# Split data

In [ ]:
# Split into train_validate and test sets
train_validate, test = train_test_split(df, test_size=.2, random_state=123)

# Split into train and validate sets
train, validate = train_test_split(train_validate, test_size=.25, random_state=123)

